Generates the circuits used for the main results in the paper. These are already generated and stored in circuits/SD and circuits/SI however they can be regenerated here (and are placed in circuits/supplementary_circuits/example_circuits)

In [2]:
import sinter
import stim
import numpy as np

import os
import sys

import shutil
import subprocess

sys.path.insert(0, 'functions') 
import oscAntgencircuitv2 as oscant
import morefuncs as morefuncs
import com_funcs5 as funcs

# Update with any changes to these files:
from importlib import reload
reload(morefuncs)
reload(funcs)
reload(oscant);

In [3]:
reload(morefuncs)


directory = "circuits/supplementary_circuits/example_circuits/"

# Delete existing circuits (so as not to simulate them)
shutil.rmtree(f'{directory}/SD/', ignore_errors=True)
shutil.rmtree(f'{directory}/SI/', ignore_errors=True)

# Make directories to store new circuits:
for path in ['SI/x', 'SI/z', 'SD/x', 'SD/z']:
    os.makedirs(f'{directory}/{path}', exist_ok=True)


# Generate circuits:

# x = (...) and z = (...) below define the CNOT order for x and z type stabilisers. E.g. order 10231203 avoids hook errors in the rotated code and is a worst-case CNOT order for the unrotated code if doing memory Z experiments (worst-case taken to be most applicable to all states). 

p_values = [0.0005,0.0007,0.001,0.0015,0.002,0.003,0.004,0.0049,0.005,0.0051,0.0052,0.0055,0.0056,0.0057,0.0058,0.006,0.007,0.008,0.009,0.01] 

# CNOT orders for x-type and z-type stabiliser measurements:
x = (1,0,2,3) # x hook error in rotated code if last two x-type CNOT's are 12, 21, 03 or 30
z = (1,2,0,3) # z hook error in rotated code if last two z-type CNOT's are 01, 10, 23 or 32

for ro in ['unro','ro']:
    
    if ro == 'unro':
        unro_CNOT_order = int(''.join(map(str, x + z)))
        ds = range(2,18)

    elif ro == 'ro':
        ro_CNOT_order = int(''.join(map(str, x + z)))
        ds = range(2,23)
    
    for d in ds:
        for p in p_values:
            if ro == 'unro':
                if d > 13 and p < 0.0007:
                    continue
                if d > 15 and p < 0.001:
                    continue
            
            if ro == 'ro':
                if d > 17 and p < 0.0007:
                    continue
                if d > 18 and p < 0.001:
                    continue

            
            for b in 'xz':
                thecircuit = oscant.generate_circuit(
                    f"surface_code:{ro}tated_memory_{b}",
                    rounds=3*d, 
                    distance=d, 
                    after_clifford_depolarization=p,
                    after_reset_flip_probability=p,
                    before_measure_flip_probability=p, 
                    # before_round_data_depolarization=p, # can be removed as is then implemented by adding idling errors function anyway
                    x_cnot_order = x,
                    z_cnot_order = z,
                    exclude_other_basis_detectors = True, # no need to add their detections to the graph which will be given to PyMatching as it is not a correlated decoder (still perform the opposite basis' stabiliser measurements though)
                )

                morefuncs.add_idling_errors_and_save_circuit(thecircuit, b, d, p, ro, x, z, directory) # saves circuit in circuits/SD (standard depolarising)

                # # Optional: Generate SI (SI with CX gates) p_values model circuits too by adjusting error probs. of SD circuits then saving a separate circuit:

                thecircuit = morefuncs.make_SI_circuit(b, d, p, ro, x, z, directory)